# ETL from Source to TemplateDB

In [ ]:
USE master
GO

DECLARE @errorMSG varchar(300) = ''

IF DB_ID('TemplateDB') IS NULL
   SET @errorMSG = @errorMSG +  'TemplateDB missing' + CHAR(13)
IF DB_ID('MinistryUniversityDWH') IS NULL
   SET @errorMSG = @errorMSG +  'MinistryUniversityDWH missing' + CHAR(13)

if (@errorMSG is null or @errorMSG = '')
    print 'All databases seem good to go'
else 
    print @errorMSG

## Empty and reset DWH

In [ ]:
USE MinistryUniversityDWH
GO

DELETE FROM [stage].[Stage_DWH_Data];

DBCC CHECKIDENT ('[stage].[Stage_DWH_Data]', RESEED, 0);


In [ ]:
-- Reset data mart if needed
USE MinistryUniversityDWH
GO

DELETE FROM [dwh].[Fact_ExamPerformance];
DELETE FROM [dwh].[Dim_Course];
DELETE FROM [dwh].[Dim_Student];
DELETE FROM [dwh].[Dim_Study];
DELETE FROM [dwh].[Dim_Date];

DBCC CHECKIDENT ('[dwh].[Fact_ExamPerformance]', RESEED, 0);
DBCC CHECKIDENT ('[dwh].[Dim_Course]', RESEED, 0);
DBCC CHECKIDENT ('[dwh].[Dim_Student]', RESEED, 0);
DBCC CHECKIDENT ('[dwh].[Dim_Study]', RESEED, 0);
DBCC CHECKIDENT ('[dwh].[Dim_Date]', RESEED, 0);

## Insert data

### To staging

In [ ]:
USE MinistryUniversityDWH

INSERT INTO [stage].Stage_DWH_Data
([university], [institute], [course_name], [year], [semester], [month], [gender], [country], [city], [faculty], [study_program], [grade], [success], [ects])
SELECT u.Name_NK, c.Institute, c.course_name, e.[Year], e.Semester, e.[Month], s.Gender, s.Country, s.City, st.Faculty, st.study_program, e.Grade, (Case When e.Grade <= 4 THEN 1 Else 0  END) as 'Success', c.ECTS
FROM [TemplateDB].[dbo].[Registration] r
JOIN [TemplateDB].[dbo].[Course] c ON r.Course_SK = c.Course_SK
JOIN [TemplateDB].[dbo].[Exam] e ON r.Exam_SK = e.Exam_SK
JOIN [TemplateDB].[dbo].[Student] s ON r.Student_SK = s.Student_SK
JOIN [TemplateDB].[dbo].[Study] st ON r.Study_SK = st.Study_SK
JOIN [TemplateDB].[dbo].[University] u ON u.University_ID = c.Source_ID AND u.University_ID = e.Source_ID AND u.University_ID = s.source_id AND u.University_ID = st.Source_ID

### Fill dimension tables

In [ ]:
USE MinistryUniversityDWH

INSERT INTO dwh.Dim_Student
([gender], [city], [country])
SELECT DISTINCT s.Gender, s.City, s.Country
FROM TemplateDB.dbo.Student s

INSERT INTO dwh.Dim_Course
([university], [institute], [course_name])
SELECT DISTINCT u.Name_NK, c.Institute, c.course_name
FROM TemplateDB.dbo.Course c
JOIN TemplateDB.dbo.University u ON c.Source_ID = u.University_ID

INSERT INTO dwh.Dim_Study
([Faculty], [study_program])
SELECT DISTINCT st.Faculty, st.study_program
FROM TemplateDB.dbo.Study st

INSERT INTO dwh.Dim_Date
([Year], [Semester], [Month])
SELECT DISTINCT e.[Year], e.[Semester], e.[Month]
FROM TemplateDB.dbo.Exam e

### Get IDs from DWH

In [ ]:
USE MinistryUniversityDWH

UPDATE sdd
SET sdd.student_DWH_ID = ds.Dim_Student_ID
FROM [stage].[Stage_DWH_Data] sdd
JOIN dwh.Dim_Student ds
    ON sdd.Gender = ds.Gender AND sdd.City = ds.City AND sdd.Country = ds.Country

UPDATE sdd
SET sdd.Course_DWH_ID = dc.Dim_Course_ID
FROM [stage].[Stage_DWH_Data] sdd
JOIN dwh.Dim_Course dc
    ON sdd.University = dc.University AND sdd.Institute = dc.Institute AND sdd.course_name = dc.course_name

UPDATE sdd
SET sdd.Study_DWH_ID = dst.Dim_Study_ID
FROM [stage].[Stage_DWH_Data] sdd
JOIN dwh.Dim_Study dst
    ON sdd.Faculty = dst.Faculty AND sdd.study_program = dst.study_program

UPDATE sdd
SET sdd.Date_DWH_ID = dd.Dim_Date_ID
FROM [stage].[Stage_DWH_Data] sdd
JOIN dwh.Dim_Date dd
    ON sdd.[Year] = dd.[Year] AND sdd.Semester = dd.Semester AND sdd.[Month] = dd.[Month] 

### Populate fact table

In [ ]:
USE MinistryUniversityDWH

INSERT INTO dwh.Fact_ExamPerformance
(dim_student_id, dim_Course_id, dim_study_id, dim_date_id, grade, success, ects)
SELECT d.student_dwh_id, d.course_dwh_id, d.study_dwh_id, d.date_dwh_id, d.grade, d.success, d.ects
FROM stage.Stage_DWH_Data d

In [ ]:
USE MinistryUniversityDWH

SELECT ds.Gender, ds.City, ds.Country, dc.University, dc.Institute, dc.course_name, fep.ECTS, fep.Grade, dst.study_program, dd.[Year], dd.Semester,  dd.[Month]
FROM [dwh].[Fact_ExamPerformance] fep
JOIN [dwh].[Dim_Student] ds ON ds.Dim_Student_ID = fep.dim_student_id
JOIN [dwh].[Dim_Course] dc ON dc.Dim_Course_ID = fep.dim_Course_id
JOIN [dwh].[Dim_Study] dst ON dst.Dim_Study_ID = fep.dim_study_id
JOIN [dwh].[Dim_Date] dd ON dd.Dim_Date_ID = fep.dim_date_id